In [1]:
import sqlite3
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats
import seaborn as sns
import datetime
from collections import Counter
import models
import math

import warnings
warnings.filterwarnings('ignore')

In [2]:
# Connect to airbnb database
connection = sqlite3.connect('airbnb.db')

In [3]:
# Query the listings table in airbnb.db and convert the listings table to dataframe
vwListings = pd.read_sql('SELECT * FROM most_recent_listings', connection)

Since the original data set contains a lot of noise, we will clean it up before feeding to the model. After completing EDA, we decided to drop some observations, convert data types, and manipulate some variables.

In [4]:
## STEVE: convert price to numeric
vwListings['price'] = vwListings['price'].str.replace(',','',regex=False)
vwListings['price'] = vwListings['price'].str.strip('$').astype('float')

In [5]:
## STEVE: host_reponse_rate, host_acceptance_rate - fillna ('') with 0
# Create room_type_mod grouping hotel rooms together
vwListings['room_type_mod'] = vwListings['room_type']
vwListings.loc[vwListings['property_type'].isin(['Room in hotel', 'Room in boutique hotel',
        'Private room in bed and breakfast']), 'room_type_mod'] = 'Hotel room'

# Drop the listings having price = 0
vwListings.drop(vwListings.loc[vwListings['price']==0].index, inplace=True)

# Drop the unavailable listings
vwListings.drop(vwListings.loc[vwListings['has_availability']=='f'].index, inplace=True)
vwListings = vwListings.reset_index(drop=True)

# Replace null reviews_per_month = 0
vwListings['reviews_per_month'] = vwListings['reviews_per_month'].replace({'': 0})

# Drop 2 listings having over 31 reviews per month
vwListings.drop(vwListings.loc[vwListings['reviews_per_month'] > 31].index, inplace=True)
vwListings = vwListings.reset_index(drop=True)

# Replace null year in host_since by the most common value 2019-09-13
vwListings['host_since'] = vwListings['host_since'].replace({'': '2019-09-13'})
vwListings['host_since'] = pd.to_datetime(vwListings['host_since'])

# Replace null value by 'N/A' in host_response_time
vwListings['host_response_time'] = vwListings['host_response_time'].replace({'': 'N/A'})

# Replace N/A value in host_response_rate = 0
vwListings['host_response_rate'] = vwListings['host_response_rate'].str.replace('N/A', '0%')
vwListings['host_response_rate'] = vwListings['host_response_rate'].str.replace('%', '')
vwListings['host_response_rate'] = pd.to_numeric(vwListings['host_response_rate'])
vwListings['host_response_rate'] = vwListings['host_response_rate'].fillna(0)

# Replace N/A value in host_acceptance_rate = 0
vwListings['host_acceptance_rate'] = vwListings['host_acceptance_rate'].str.replace('N/A', '0%')
vwListings['host_acceptance_rate'] = vwListings['host_acceptance_rate'].str.replace('%', '')
vwListings['host_acceptance_rate'] = pd.to_numeric(vwListings['host_acceptance_rate'])
vwListings['host_acceptance_rate'] = vwListings['host_acceptance_rate'].fillna(0)

# Replace null value by 'f' in host_is_superhost
vwListings['host_is_superhost'] = vwListings['host_is_superhost'].replace({'': 'f'})

# Replace null value by 'f' in host_verifications
vwListings['host_verifications'] = vwListings['host_verifications'].replace({'': 'f'})

# Replace null value by 'f' in host_has_profile_pic
vwListings['host_has_profile_pic'] = vwListings['host_has_profile_pic'].replace({'': 'f'})

# Replace null value by 'f' in host_identity_verified
vwListings['host_identity_verified'] = vwListings['host_identity_verified'].replace({'': 'f'})

Next, we will clean the bedrooms variable. We will follow the rules below:
- If the value of the bedrooms variable is null and the value of the beds variable is not null, bedrooms will be equal to the value of beds.
- If the value of the beds variable is null and the value of the bedrooms variable is not null, beds will be equal to the value of bedrooms.
- Otherwise, both of the variables will be equal the mean of corresponding variable.

In [6]:
## STEVE: 
# clean up bedrooms before baths since baths depends on bedrooms
# clean up code, convert to numeric
vwListings.loc[(vwListings['bedrooms'] == '')&(vwListings['beds'] == ''), ['bedrooms', 'beds']] = [2, 2]
vwListings.loc[vwListings['bedrooms'] == '', 'bedrooms'] = vwListings.loc[vwListings['bedrooms'] == '', 'beds']
vwListings.loc[vwListings['beds'] == '', 'beds'] = vwListings.loc[vwListings['beds'] == '', 'bedrooms']

vwListings['bedrooms'] = pd.to_numeric(vwListings['bedrooms'])
vwListings['beds'] = pd.to_numeric(vwListings['beds'])

We will convert some bathroom text to the number of bathrooms based on the human's interpretation. With the null values or the half bathroom only, we will assume the number of bathrooms is equal to the mean bathrooms which is 1.

In [7]:
## STEVE: clean up code
vwListings.loc[vwListings['bathrooms_text'] == '0 shared baths', 'bathrooms_text'] = \
    vwListings.loc[vwListings['bathrooms_text'] == '0 shared baths', 'bedrooms'].astype('str') + ' baths'
vwListings['bathrooms_text'] = vwListings['bathrooms_text'].replace(
    ['0 baths', 'Half-bath', 'Private half-bath', 'Shared half-bath', ''], '1 bathroom')

In [8]:
## STEVE: change to get half baths in
# Extract the number of bathrooms from bathroom_text.
#vwListings['bathroom_numbers'] = vwListings['bathrooms_text'].str.extract('(\d+)') # previous
vwListings['bathroom_numbers'] = vwListings['bathrooms_text'].str.split().str.get(0)
vwListings['bathroom_numbers'] = pd.to_numeric(vwListings['bathroom_numbers'])

In [9]:
# This step is to prepare for One Hot Encodings
for i in range(len(vwListings['host_response_time'])):
    if 'within an hour' in vwListings['host_response_time'][i]:
        vwListings['host_response_time'][i] = 'within_an_hour'
    elif 'N/A' in vwListings['host_response_time'][i]:
        vwListings['host_response_time'][i] = 'na'
    elif 'within a few hours' in vwListings['host_response_time'][i]:
        vwListings['host_response_time'][i] = 'within_a_few_hours'
    elif 'within a day' in vwListings['host_response_time'][i]:
        vwListings['host_response_time'][i] = 'within_a_day'
    elif 'a few days or more' in vwListings['host_response_time'][i]:
        vwListings['host_response_time'][i] = 'a_few_days_or_more'

        
for i in range(len(vwListings['room_type_mod'])):
    if 'Entire home/apt' in vwListings['room_type_mod'][i]:
        vwListings['room_type_mod'][i] = 'entire_home_apt'
    elif 'Private room' in vwListings['room_type_mod'][i]:
        vwListings['room_type_mod'][i] = 'private_room'
    elif 'Hotel room' in vwListings['room_type_mod'][i]:
        vwListings['room_type_mod'][i] = 'hotel_room'
    elif 'Shared room' in vwListings['room_type_mod'][i]:
        vwListings['room_type_mod'][i] = 'shared_room'

In [10]:
# Convert the all the review scores to numeric
vwListings['review_scores_rating'] = pd.to_numeric(vwListings['review_scores_rating'])
vwListings['review_scores_accuracy'] = pd.to_numeric(vwListings['review_scores_accuracy'])
vwListings['review_scores_cleanliness'] = pd.to_numeric(vwListings['review_scores_cleanliness'])
vwListings['review_scores_checkin'] = pd.to_numeric(vwListings['review_scores_checkin'])
vwListings['review_scores_communication'] = pd.to_numeric(vwListings['review_scores_communication'])
vwListings['review_scores_location'] = pd.to_numeric(vwListings['review_scores_location'])
vwListings['review_scores_value'] = pd.to_numeric(vwListings['review_scores_value'])

In [11]:
## STEVE: fill review_scores na with the mean
for col in vwListings.columns:
    if col.startswith('review_scores'):
        x = vwListings[col]
        vwListings[col] = vwListings[col].fillna(x.mean())

In [12]:
#  the coordinates (latitude, longtitute of a place from google map )
smithsonian_lat = 38.8889532077163
smithsonian_long = -77.0259992316524
lincoln_memorial_lat = 38.8893771334115
lincoln_memorial_long = -77.0501761044084
capitol_lat = 38.8900557831996
capttol_long = -77.0090290467363
white_house_lat = 38.9103145107623
white_house_long = -77.0221504467353
library_of_congress_lat = 38.8888008852652
library_of_congress_long = -77.0047082755724
nationals_park_lat = 38.8731981381059
nationals_park_long = -77.0074329
national_zoo_lat = 38.9294590679026
national_zoo_long = -77.0492384835061
union_station_lat = 38.8973065709087
union_station_long = -77.00629767372
mcpherson_sq_lat = 38.9028682064423
mcpherson_sq_long = -77.0324745881233

In [13]:
# distance unit = miles
vwListings["distance_smithsonian"] = np.arccos(np.sin(np.radians(vwListings["latitude"]))
                                            * np.sin(np.radians(smithsonian_lat)) 
                                            + np.cos(np.radians(vwListings["latitude"]))
                                            *np.cos(np.radians(smithsonian_lat))
                                            *np.cos(np.radians(smithsonian_long-vwListings["longitude"])) )*3963*1.15077945

In [14]:
vwListings["distance_lincoln"] = np.arccos(np.sin(np.radians(vwListings["latitude"]))
                                            * np.sin(np.radians(lincoln_memorial_lat)) 
                                            + np.cos(np.radians(vwListings["latitude"]))
                                            *np.cos(np.radians(lincoln_memorial_lat))
                                            *np.cos(np.radians(lincoln_memorial_long-vwListings["longitude"])) )*3963*1.15077945

In [15]:
vwListings["distance_capitol"] = np.arccos(np.sin(np.radians(vwListings["latitude"]))
                                            * np.sin(np.radians(capitol_lat)) 
                                            + np.cos(np.radians(vwListings["latitude"]))
                                            *np.cos(np.radians(capitol_lat))
                                            *np.cos(np.radians(capttol_long-vwListings["longitude"])) )*3963*1.15077945

In [16]:
vwListings["distance_white_house"] = np.arccos(np.sin(np.radians(vwListings["latitude"]))
                                            * np.sin(np.radians(white_house_lat)) 
                                            + np.cos(np.radians(vwListings["latitude"]))
                                            *np.cos(np.radians(white_house_lat))
                                            *np.cos(np.radians(white_house_long-vwListings["longitude"])) )*3963*1.15077945

In [17]:
vwListings["distance_library_congress"] = np.arccos(np.sin(np.radians(vwListings["latitude"]))
                                            * np.sin(np.radians(library_of_congress_lat)) 
                                            + np.cos(np.radians(vwListings["latitude"]))
                                            *np.cos(np.radians(library_of_congress_lat))
                                            *np.cos(np.radians(library_of_congress_long-vwListings["longitude"])))*3963*1.15077945

In [18]:
vwListings["distance_nationals_park"] = np.arccos(np.sin(np.radians(vwListings["latitude"]))
                                            * np.sin(np.radians(nationals_park_lat)) 
                                            + np.cos(np.radians(vwListings["latitude"]))
                                            *np.cos(np.radians(nationals_park_lat))
                                            *np.cos(np.radians(nationals_park_long-vwListings["longitude"])) )*3963*1.15077945

In [19]:
vwListings["distance_national_zoo"] = np.arccos(np.sin(np.radians(vwListings["latitude"]))
                                            * np.sin(np.radians(national_zoo_lat)) 
                                            + np.cos(np.radians(vwListings["latitude"]))
                                            *np.cos(np.radians(national_zoo_lat))
                                            *np.cos(np.radians(national_zoo_long-vwListings["longitude"])) )*3963*1.15077945

In [20]:
vwListings["distance_railway"] = np.arccos(np.sin(np.radians(vwListings["latitude"]))
                                            * np.sin(np.radians(union_station_lat)) 
                                            + np.cos(np.radians(vwListings["latitude"]))
                                            *np.cos(np.radians(union_station_lat))
                                            *np.cos(np.radians(union_station_long-vwListings["longitude"])) )*3963*1.15077945

In [21]:
vwListings["distance_mcpherson"] = np.arccos(np.sin(np.radians(vwListings["latitude"]))
                                            * np.sin(np.radians(mcpherson_sq_lat)) 
                                            + np.cos(np.radians(vwListings["latitude"]))
                                            *np.cos(np.radians(mcpherson_sq_lat))
                                            *np.cos(np.radians(mcpherson_sq_long-vwListings["longitude"])) )*3963*1.15077945

We will convert one hot encodings for the following categorical variables host_response_time, host_is_superhost, host_has_profile_pic, host_identity_verified, room_type_mod

In [22]:
## STEVE: add instant_bookable dummies
vwListings = pd.concat([vwListings, pd.get_dummies(vwListings['host_response_time'], prefix='response_time', prefix_sep='_')], axis=1)
vwListings = pd.concat([vwListings, pd.get_dummies(vwListings['host_is_superhost'], prefix='superhost', prefix_sep='_')], axis=1)
vwListings = pd.concat([vwListings, pd.get_dummies(vwListings['host_identity_verified'], prefix='identity_verified', prefix_sep='_')], axis=1)
vwListings = pd.concat([vwListings, pd.get_dummies(vwListings['host_has_profile_pic'], prefix='profile', prefix_sep='_')], axis=1)
vwListings = pd.concat([vwListings, pd.get_dummies(vwListings['room_type_mod'])], axis=1)
vwListings = pd.concat([vwListings, pd.get_dummies(vwListings['instant_bookable'], prefix='instant_bookable', prefix_sep='_')], axis=1) # ADDED

In [23]:
## STEVE: host_name == "June" stuff
# For many of their private room listings with only 1 bedroom available, they count bedrooms on the entire property
# correcting max_nights < min_nights
vwListings.loc[(vwListings['host_id']==294545484)&(vwListings['room_type_mod']=='private_room'), 'bedrooms'] = 1
vwListings.loc[vwListings['maximum_nights'] < vwListings['minimum_nights'], 'maximum_nights'] = vwListings.loc[
    vwListings['maximum_nights'] < vwListings['minimum_nights'], 'minimum_nights']

In [24]:
# STEVE: lower egregiously high maximum_nights
vwListings.loc[vwListings['maximum_nights'] > 1125, 'maximum_nights'] = 1125

### MODEL 1

In [25]:
# NEED TO CONFIRM WITH STEVE AGAIN
# Convert the date time of last review to numeric
#vwListings['last_review'] = pd.to_datetime(vwListings['last_review'])

In [26]:
## STEVE: take out host_since (temporary)
# TEMPORARY TAKE OUT last_review 
model = ("price ~ response_time_a_few_days_or_more + response_time_within_a_day "
        "+ response_time_within_a_few_hours + response_time_within_an_hour + host_response_rate "
        "+ host_acceptance_rate + superhost_t "
        "+ profile_t + identity_verified_t + entire_home_apt + hotel_room + private_room + accommodates "
        "+ bathroom_numbers + bedrooms + beds + minimum_nights + maximum_nights + number_of_reviews "
        "+ review_scores_rating + review_scores_accuracy + review_scores_cleanliness + review_scores_checkin "
        "+ review_scores_communication + review_scores_location + review_scores_value + instant_bookable_t " # instant_bookable_t
        "+ calculated_host_listings_count + distance_smithsonian + distance_lincoln + distance_capitol "
        "+ distance_white_house + distance_library_congress + distance_nationals_park + distance_national_zoo "
        "+ distance_railway + distance_mcpherson")
result = models.bootstrap_linear_regression(model, data=vwListings)
models.describe_bootstrap_lr(result, sd=4)

,,,95% BCI</strong,
Coefficients</strong,,Mean</strong,Lo</strong,Hi</strong
,$\beta_{0}$,-226.9126,-298.4979,-162.3717
response_time_a_few_days_or_more,$\beta_{1}$,-7.1476,-37.4401,30.0487
response_time_within_a_day,$\beta_{2}$,-18.3310,-96.9465,57.5824
response_time_within_a_few_hours,$\beta_{3}$,-45.5519,-115.0615,19.4849
response_time_within_an_hour,$\beta_{4}$,-12.2299,-85.9788,56.6449
host_response_rate,$\beta_{5}$,-0.0368,-0.8209,0.7333
host_acceptance_rate,$\beta_{6}$,0.1855,-0.0444,0.3876
superhost_t,$\beta_{7}$,8.9451,-1.2320,17.8648
profile_t,$\beta_{8}$,6.3869,-18.2522,29.6138
